In [9]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [10]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df['Azimuth'].values

    return X, y, df

In [11]:
csv_path = 'cleaned/cleaned_data_quartal_1_2020.csv'
X, y, df = load_dataset(csv_path)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [13]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

In [14]:
timesteps = 1

In [15]:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [16]:
model = Sequential()
model.add(LSTM(units=2048, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=512, activation= 'relu', return_sequences=True))
model.add(LSTM(units=256, activation= 'relu', return_sequences=True))
model.add(LSTM(units=128, activation= 'relu', return_sequences=True))
model.add(LSTM(units=64, activation= 'relu', return_sequences=True))
model.add(LSTM(units=32, activation= 'relu'))
model.add(Dense(units=1))

optimizer = Adam(learning_rate= 0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])

history = model.fit(X_train_reshaped, y_train_scaled, epochs= 500, batch_size = 512,  validation_data=(X_test_reshaped, y_test_scaled))

Epoch 1/500
21/21 [==============================] - 7s 86ms/step - loss: 0.2897 - accuracy: 9.5383e-05 - val_loss: 0.2860 - val_accuracy: 0.0000e+00
Epoch 2/500
21/21 [==============================] - 1s 51ms/step - loss: 0.2859 - accuracy: 9.5383e-05 - val_loss: 0.2821 - val_accuracy: 0.0000e+00
Epoch 3/500
21/21 [==============================] - 1s 49ms/step - loss: 0.2819 - accuracy: 9.5383e-05 - val_loss: 0.2781 - val_accuracy: 0.0000e+00
Epoch 4/500
21/21 [==============================] - 1s 49ms/step - loss: 0.2778 - accuracy: 9.5383e-05 - val_loss: 0.2739 - val_accuracy: 0.0000e+00
Epoch 5/500
21/21 [==============================] - 1s 48ms/step - loss: 0.2734 - accuracy: 9.5383e-05 - val_loss: 0.2694 - val_accuracy: 0.0000e+00
Epoch 6/500
21/21 [==============================] - 1s 50ms/step - loss: 0.2686 - accuracy: 9.5383e-05 - val_loss: 0.2642 - val_accuracy: 0.0000e+00
Epoch 7/500
21/21 [==============================] - 1s 51ms/step - loss: 0.2626 - accuracy: 9.5383e

In [17]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)
accuracy = model.evaluate(X_test_reshaped, y_test_scaled)

82/82 [==============================] - 1s 9ms/step - loss: 1.3794e-06 - accuracy: 0.0000e+00


In [23]:
timestamps_test = X_test[:, 0]
df_results = pd.DataFrame({
    'Timestamp': timestamps_test,
    'Actual_Azimuth': y_test,
    'Predicted_Azimuth': predicted_values.flatten()
})

In [ ]:
output_csv_path = 'azimuth_prediction_results.csv'
df_results.to_csv(output_csv_path, index=False)